# MPNN for edge betweenness

The notebook will train a model to compute edge betweenness of edges of a graph, in an inductive setting allowing for generalization to unseen graphs.
This model will be used to predict edge betweenness inside the Girvan-Newman algorithm by networkx library.
Steps:
1. get a dataset to detect community in
2. train an edge classifier (not node) for edge betweenness
3. save this model to disk
4. load this model and predict edge betweenness on unseen graphs
5. use networkx Girvan-Newman algorithm with a replacement for best edge selection routine, a replacmenet that loads the model and predict edge betweenness

# Summary

# 1 Dataset

## 1.1 datasets available:
multiple graphs in wihch we can compute edge betweenness
Choice: MUTAG, ENZYMES, PROTEINS...

## 1.2 target:
we need the edge betweenness of each edge,  which we can compute with networkx for example

## 1.3 dataset creation
1. get the graphs from MUTAG
2. transform to networkx
3. compute the edge betweenness
4. save to networkx graph format
5. transform to PyTorch_Geometric dataset
6. pass this dataset to model training task

# 2 Model 

## 2.1 Choice of the graph neural network
Choose or create a GNN that 
1. considers and __predicts__ edge features!
2. works in an inductive setting (predicts on unseen graphs)

Ideas:
- Link prediction algorithm
- Link prediction algorithm modified?
- MPNN
- GraphSAGE modified?
- Graph Network (DeepMind)

models allowing edge features:
- GNN
- MPNN
- DCNN
- PATCHY-SAN
- DGCNN [Deep Graph Convolutional Neural Network](https://www.groundai.com/project/link-prediction-based-on-graph-neural-networks/)
- EGNNA

other ideas:
- model modif -> predict values edges
    - betweenness associated to shortedst path -> features nodes not needed for example
    - take a look at link prediction algorithms..
- maybe node features are also useful..
    - set a feature vector with a real value for each edge of a node
    - predict that feature vector **STRANGE Multiple regression..**
- change the model:
    - edges with their associated edge neighbors -> edge adjacency matrix
    - COMBINE function combines ehv + ehn n in EN(v) EN(v)="Edge Neighborhood of edge v"
- additional node in each edge
- transform nodes to edges **IMPOSSIBLE?**
- min betweens in both vertex=edge betweenness **WRONG**
 

## 2.2 Implementation
Available implementation or build. PyTorchGeometric seems a good tool to implement models..


## 2.3 Hyperparameter search
Which hyperparams to search for optimal performance?

# 3 Modify Girvan-Newman

Modify the Girvan-Newman algorithm by calling a new function that:
    1. loads the GNN model from disk
    2. predicts the edge betweenness of all edges of a graph
    3. ranks the edges according to that edge betweenness
The rest is already taken car by networkx.

Time the improvement: time original and new Girvan-Newman execution and see difference in times

# Implementation

## 1 Dataset

In [3]:
import importlib
import torch
from torch_geometric.data import DataLoader
import networkx as nx
from torch_geometric.data import Data

def loadDataset(collection, name=None):
    # import datasets
    themodule = importlib.import_module("torch_geometric.datasets")
    # get the function corresponding to collection
    method_to_call = getattr(themodule, collection)
    if name:
        return method_to_call(root='./data/'+str(collection), name=name)
    else:
        return method_to_call(root='./data/'+str(collection)) 

    
def writeAdjacencyMatrixToDisk(G, filename='temp_adjacency_matrix.txt'):
    """
        Transform to networkx dataset

        possible formats: GML, Adjacency matrix, ..
        start by Adjcency list 
             --> (ignoring edge/node features)
             --> line format: source target target2 target3 ... 
        later we can improve this...
    """
    f = open(filename,'w')
    _ni=-1
    newline = False
    theline = []
    careturn = ""
    for ei in range(G.edge_index.size()[1]):
        if int(G.edge_index[0,ei].item()) != _ni:
            newline=True
            _ni=int(G.edge_index[0,ei].item())
            
        else:
            newline=False
            
            
        ni = str(G.edge_index[0,ei].item())
        vi = str(G.edge_index[1,ei].item())
        if newline:
            f.write(''.join(theline))
            #print(''.join(theline))
            #print(" --> "+str(_ni))
            theline =[]
            theline.append(careturn+ni+" ")
            theline.append(vi+" ")
            careturn = "\n"
        else:
            theline.append(vi+" ")
        # print("({},{})".format(ni,vi))
    
    
def nx_createNxGraphInMem(G):
    """
        Transform to networkx dataset

        possible formats: GML, Adjacency matrix, ..
        start by Adjcency list 
             --> (ignoring edge/node features)
             --> line format: source target target2 target3 ... 
        later we can improve this...
    """
    g = nx.MultiGraph()
   
    for ei in range(G.edge_index.size()[1]):    
        ni = str(G.edge_index[0,ei].item())
        vi = str(G.edge_index[1,ei].item())
        g.add_edge(ni,vi)
    return g
    
def nx_verifyEdges(G, g):
    for ei in range(G.edge_index.size()[1]):
        ni = str(G.edge_index[0,ei].item())
        vi = str(G.edge_index[1,ei].item())
        if (ni,vi,0) not in list(g.edges):
            if (vi,ni,1) not in list(g.edges):
                print("Error {} not in networkx graph".format((ni,vi)))
            
        

def nx_compute_edge_betweenness(G):
    
    print(list(G.edges)[:10])
    G_components = nx.connected_component_subgraphs(G)
    G_mc = list(G_components)[0]  
    eb_dict_res = {}
    eb_dict = nx.edge_betweenness_centrality(G_mc)
    
    # if there are more connected components...
    if len(list(G_components))>1:
        print("connected components: ",len(list(G_components)))
    
    eb_dict_res.update(eb_dict)
    
        
    return eb_dict_res


def update_edge_betweenness(G, eb_dict):
    """
        G.edge_attr must contain the edge betweenness values 
        for each edge
        
        G.y must contain it also.. (it is a copy of the edge betweenness..)
        this could help the training phase
        
        Size restrictions:
        - Given the size of the graphs, is it better to just transform the 
        object instead to write a new one?
        - also just use G.y? but for GNN algorithms..not sure
        
        new_edg_attr will be size [num edges, 1]
        and must be sorted in accordance to G.edge_index
        
    
    """
    
    new_edg_attr = []
    for i in range(len(G.edge_index[0])):
        ni = G.edge_index[0][i]
        vi = G.edge_index[1][i]
        
        if ni and vi:
            ni=str(ni.item())
            vi=str(vi.item())
            #print((ni,vi))
            try:
                new_edg_attr.append([eb_dict[(ni,vi)]])
            except:
                try:
                    new_edg_attr.append([eb_dict[(vi,ni)]])
                except:
                    #print("ERROR {} and {} not found!".format((ni,vi),(vi,ni)))
                    new_edg_attr.append([0])
        else:
            new_edg_attr.append([0])

    new_edg_attr = torch.FloatTensor(new_edg_attr)
    
    #newG = Data(
    #    x=G.x, 
    #    edge_index=G.edge_index, 
    #    edge_attr=new_edg_attr,
    #    y=new_edg_attr)
    
    #G.edge_attr = new_edg_attr
    G.y = new_edg_attr
    
    return G

In [4]:

from torch_geometric.data import DataLoader
import networkx as nx

# load the dataset examples---------------------------------------
#PPI
#dataset = loadDataset('PPI')
#QM7b
#dataset = loadDataset('QM7b')
#MUTAG
#dataset = loadDataset(collection='Entities',name='MUTAG')
#ENZYMES FROM TUDataset
dataset = loadDataset(collection='TUDataset',name='ENZYMES')

print(dataset)
print("\n data: ",dataset.data) # many graphs!
#print("\n num_features: ",dataset.num_features)
#print("\n num_classes: ",dataset.num_classes)

# read graphs------------------------------------------------------
loader = DataLoader(dataset, batch_size=32, shuffle=False)


i = 0
prefix = 'temp_aj_m'
for G in loader:
    print(G)
    
    # Transform to networkx graph-----------------------------------
    # write to adjacency matrix on disk
    writeAdjacencyMatrixToDisk(G, filename=prefix+str(i)+'.txt')
    
    # load into a networkx graph object
    g2 = nx.read_adjlist(prefix+str(i)+'.txt')
    #g2 = nx_createNxGraphInMem(G)
    
    # compute edge betweenness--------------------------------------
    eb_dict = nx_compute_edge_betweenness(g2)
    
    # write edge betweenness back to PyTorch Geometric graph--------
    # 6 edge_betweenneess are missing
    G = update_edge_betweenness(G,eb_dict)
    
    #print(dir(G))
    print(G.y)
    print(sum(G.y)) # so G.y is not all zeroes
    print(G.edge_index)
    print(G)
    
    i+=1
    if i==1:
        break
    

    

ENZYMES(600)

 data:  Data(edge_index=[2, 74564], x=[19580, 3], y=[600])
Batch(batch=[1069], edge_index=[2, 4264], x=[1069, 3], y=[32])
[('0', '1'), ('0', '2'), ('0', '3'), ('1', '2'), ('1', '3'), ('1', '24'), ('1', '27'), ('2', '3'), ('2', '27'), ('2', '28')]
tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]])
tensor([9.8498])
tensor([[   0,    0,    0,  ..., 1068, 1068, 1068],
        [   1,    2,    3,  ..., 1051, 1052, 1067]])
Batch(batch=[1069], edge_index=[2, 4264], x=[1069, 3], y=[4264, 1])


In [ ]:
# Encapsulate these functionalities into a Python module

## 2 Model

Ideas:
- Link prediction algorithm
- Link prediction algorithm modified?
- MPNN
- GraphSAGE modified?
- Graph Network (DeepMind)

models allowing edge features:
- GNN
- MPNN
- DCNN
- PATCHY-SAN
- DGCNN [Deep Graph Convolutional Neural Network](https://www.groundai.com/project/link-prediction-based-on-graph-neural-networks/)
- EGNNA

other ideas:
- additional node in each edge -> **NO SENSE**
- transform nodes to edges **IMPOSSIBLE?**
- min betweens in both vertex=edge betweenness **WRONG**
- model modif -> predict values edges
    - betweenness associated to shortedst path -> features nodes not needed for example
    - take a look at link prediction algorithms..
- maybe node features are also useful..
    - set a feature vector with a real value for each edge of a node
    - predict that feature vector **STRANGE Multiple regression..**
- change the model Link Conv Network:
    - edges with their associated edge neighbors -> edge adjacency matrix
    - COMBINE function combines ehv + ehn n in EN(v) EN(v)="Edge Neighborhood of edge v"

 


### Link convolutional network idea:

#### Edge adjacency matrix: 
it's more a python dictionary, where for each edge there are other edges listed. It's symmetric
`
show by code with nx and PyTorchGeom
`
#### Edge neighborhood:
extracted from the Edge adjacency matrix

#### Embedding generation algorithm:

Adaptation from the GraphSAGE algorithm:

Initialization: 
x(u,v) contains the features of the edge.
If no features are considered? Well at least the weight of the link must be present. So in cases there is no weight associated, se x(u,v) to 1 in all edges
$$ e^{0}_{(u,v)} = x_{(u,v)} \forall (u,v) \in G $$ 
for k = 1..K do
for (u,v) in G:
    $$ e^{k}_{EN((u,v))} = AGGREGATE_k({e^{k-1}_{(w,r)}, \forall (w,r) \in EN((u,v))}) $$ 
    $$ e^{k}_{(u,v)} = \sigma(W^k·CONCAT(e^{k-1}_{(u,v)},e^{k}_{EN((u,v))} )$$
end
$$ e^{k}_{(u,v)} = e^{k}_{(u,v)}/||e^{k}_{(u,v)}||_2, \forall (u,v) \in G $$ 
end
$$ z_{(u,v)} = e^{K}_{(u,v)} \forall (u,v) \in G $$


**Thoughts**
Probably this won't work well as edge betweennes depends on "long range" interactions...
Anyways node betweenneess works well,(maybe retest it with GraphSage and test on unseen graphs
 
**Next Steps**

Quick retest of betweenneess centrality with GCN and GraphSage (retake code from igraph test for example)

Read about PyTorch GCN model.
gcn(x, edge_index), x is features matrix, edge_index is 2 lists of nodes
- how to transform that to edge feaatures and edge_adjacency
- probably add an edge to int translation list -> then everything can now be used with original PyTorch Geometric code
- x will contain edge features (only the weights?)
- edge_index will be the Edge adjacency matrix but using edge id's
- **IMPORTANT** understand how from x (node feature matrix) the output is generated! (example return F.log_softmax(x, dim=1)

### Link prediction approaches
Read about link prediction approaches and copy their approach.

## 3 Girvan-Newman modification

In [1]:
# modifying edge_betweenness
from networkx import edge_betweenness_centrality as betweenness


def predict_all_betweennesses(G):
    centrality = {}
    
    # transform networkx graph to PyTorch Geometric Dataset
    
    
    # load model from disk
    
    
    # predict all betweennesses of edges of G
    
    # save into centrality dict 
    # { (n1,n2): 0.0938, (n3,n4): 0.1230, ..}
    
    return centrality
    

def most_central_edge(G):
    centrality = predict_all_betweennesses(G)
    return max(centrality.values())

# read graph into networkx
G = nx.read_gml('graph.gml')

# add weight 1 to all edges
nx.set_edge_attributes(G, {(u,v): 1 for u, v in G.edges()},'weight')

# compute the communities by girvan_newman algorithm
comp = girvan_newman(G, most_valuable_edge=most_central_edge)

# print the results
tuple(sorted(c) for c in next(comp))

NameError: name 'nx' is not defined